# Random Forest

In [4]:
# randomForest
install.packages("randomForest")
install.packages("cvAUC")
library(randomForest)
library(cvAUC)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘gplots’, ‘ROCR’, ‘data.table’


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: ROCR

Loading required package: data.table

 

cvAUC version: 1.1.0

Notice to cvAUC users: Major speed improvements in version 1.1.0

 



In [ ]:
# Muat set data respons biner
train <- read.csv("data/higgs_train_10k.csv")
test <- read.csv("data/higgs_test_5k.csv")

# Ukuran
dim(train)
dim(test)

# Kolom
names(train)

[1] 10000    29

[1] 5000   29

[1] "response" "x1"       "x2"       "x3"       "x4"       "x5"      
 [7] "x6"       "x7"       "x8"       "x9"       "x10"      "x11"     
[13] "x12"      "x13"      "x14"      "x15"      "x16"      "x17"     
[19] "x18"      "x19"      "x20"      "x21"      "x22"      "x23"     
[25] "x24"      "x25"      "x26"      "x27"      "x28"

In [ ]:
# Identitas kolom tanggapan
ycol <- "response"

# Identifikasi kolom prediktor
xcols <- setdiff(names(train), ycol)

# Ubah respons menjadi faktor
train[,ycol] <- as.factor(train[,ycol])
test[,ycol] <- as.factor(test[,ycol])

In [ ]:
# model RF default dengan 500 pohon
set.seed(1)  # For reproducibility
system.time(model <- randomForest(x = train[,xcols], 
                                  y = train[,ycol],
                                  xtest = test[,xcols],
                                  ntree = 50))

   user  system elapsed 
  1.492   0.010   1.503 

In [ ]:
# Buat prediksi pada set data pengujian
preds <- model$test$votes[, 2]
labels <- test[,ycol]

# Hitung AUC pada set pengujian
cvAUC::AUC(predictions = preds, labels = labels)

[1] 0.767961

## Caret Method "parRF"

In [ ]:
library(caret)
library(doParallel)
library(e1071)

In [ ]:
#"parRF" model using caret
registerDoParallel(cores = 8)

model <- caret::train(x = train[,xcols], 
                      y = train[,ycol], 
                      method = "parRF",
                      preProcess = NULL,
                      weights = NULL,
                      metric = "Accuracy",
                      maximize = TRUE,
                      trControl = trainControl(), 
                      tuneGrid = NULL,
                      tuneLength = 3)

## h2o



In [ ]:
#install.packages("h2o")
library(h2o)
#h2o.shutdown(prompt = FALSE)
h2o.init(nthreads = -1)  #Mulai cluster H2O lokal menggunakan nthreads = jumlah inti yang tersedia


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /var/folders/2j/jg4sl53d5q53tc2_nzm9fz5h0000gn/T//RtmpZIIhCg/h2o_me_started_from_r.out
    /var/folders/2j/jg4sl53d5q53tc2_nzm9fz5h0000gn/T//RtmpZIIhCg/h2o_me_started_from_r.err


Starting H2O JVM and connecting: . Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 seconds 107 milliseconds 
    H2O cluster version:        3.8.2.6 
    H2O cluster name:           H2O_started_from_R_me_pvt513 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   3.56 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  8 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 (2016-05-03) 



In [ ]:
# Muat set data respons biner
train <- h2o.importFile("./data/higgs_train_10k.csv")
test <- h2o.importFile("./data/higgs_test_5k.csv")

# Ukuran
dim(train)
dim(test)

# Kolom
names(train)

  |======================================================================| 100%
  |======================================================================| 100%


[1] 10000    29

[1] 5000   29

[1] "response" "x1"       "x2"       "x3"       "x4"       "x5"      
 [7] "x6"       "x7"       "x8"       "x9"       "x10"      "x11"     
[13] "x12"      "x13"      "x14"      "x15"      "x16"      "x17"     
[19] "x18"      "x19"      "x20"      "x21"      "x22"      "x23"     
[25] "x24"      "x25"      "x26"      "x27"      "x28"

In [ ]:
# Identitas kolom tanggapan
ycol <- "response"

# Identify the predictor columns
xcols <- setdiff(names(train), ycol)

# Convert response to factor (required by randomForest)
train[,ycol] <- as.factor(train[,ycol])
test[,ycol] <- as.factor(test[,ycol])

In [ ]:
# Latih model RF default dengan 100 pohon

system.time(model <- h2o.randomForest(x = xcols,
                                      y = ycol,
                                      training_frame = train,
                                      seed = 1, #for reproducibility
                                      ntrees = 50)) 

  |======================================================================| 100%


   user  system elapsed 
  0.217   0.008   5.502 

In [ ]:
# Hitung AUC pada set data pengujian
# H2O menghitung banyak metrik kinerja model secara otomatis, termasuk AUC

perf <- h2o.performance(model = model, newdata = test)
h2o.auc(perf)

[1] 0.7698004